# SAN - Praktische Arbeit (Marvel Universe Social Network)

## Datenbeschaffung

Die Daten haben wir [hier](https://www.kaggle.com/datasets/csanhueza/the-marvel-universe-social-network) auf Kaggle gefunden. Sie bestehen aus folgenden Dateien:
- edges.csv
- nodes.csv
- hero-edge.csv

## Explorative Datenanalyse

Nachdem die Daten beschaffen wurden, sollen einfache Datenanalysen (Häufigkeitsanalyse, Statistiken, Daten-Visualisierungen...) helfen, die Daten besser zu verstehen und Unstimmigkeiten wie auch fehlende Werte in den Daten frühzeitig zu erkennen. Passen Sie wenn nötig ihre Programme zur Datensammlung und Daten-Transformationen an, wenn Sie Probleme mit der Datenqualität erkennen. Prüfen Sie auch (wenn möglich), ob die eingesammelten Daten plausibel sindund mit der Datenquelle / Realität (z.B: anhand einer Stichprobe) übereinstimmen. 

Mögliche Probleme: Dieser Schritt ist wichtig, ist aber schlussendlich im Bewertungsraster im Vergleich zu anderen Kriterien nur schwach gewichtet. Ich empfehle die Nutzung einer Python-Library wie z.B. Pandas Profiling oder Sweetviz, welche das Leben für die explorative Datenanalyse erheblich vereinfacht. Soziale Netzwerkanalyse In der Analysephase das das eingesammelte Netzwerk / die eingesammelten Netzwerke analysiert. Im Meilenstein haben Sie sich überlegen, welche Analysen Sie auf dem Netzwerk ausführen möchten. Es sollten verschiedene Analysen aus dem SAN-Unterricht sein. Sie dürfen aber gerne auch nicht im Unterricht behandelte Netzwerk-Analysen durchführen. Es ist auch gut möglich, dass Sie plötzlich im zweiten Teil noch weitere Ideen für Analysen haben, die Sie nicht im Meilenstein erwähnt haben.Diese dürfen Sie gerne umsetzen.

In [3]:
import os
import sys
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
from pandas_profiling import ydata_profiling
import sweetviz as sv

from helper import *

## Daten einlesen und betrachten

In [4]:
df_nodes = pd.read_csv('Daten/nodes.csv')
df_edges = pd.read_csv('Daten/edges.csv')

print(df_nodes.head())
print(df_edges.head())

                   node   type
0               2001 10  comic
1                2001 8  comic
2                2001 9  comic
3  24-HOUR MAN/EMMANUEL   hero
4  3-D MAN/CHARLES CHAN   hero
                   hero   comic
0  24-HOUR MAN/EMMANUEL  AA2 35
1  3-D MAN/CHARLES CHAN   AVF 4
2  3-D MAN/CHARLES CHAN   AVF 5
3  3-D MAN/CHARLES CHAN   COC 1
4  3-D MAN/CHARLES CHAN  H2 251


In [5]:
# Attribute der Knoten und Kanten identifizieren
node_attributes = df_nodes.columns.tolist()
edge_attributes = df_edges.columns.tolist()

node_attributes, edge_attributes

(['node', 'type'], ['hero', 'comic'])

In [6]:
# Anzahl der einzigartigen Helden und Comics ermitteln
unique_heroes = df_nodes[df_nodes['type'] == 'hero']['node'].nunique()
unique_comics = df_nodes[df_nodes['type'] == 'comic']['node'].nunique()

unique_heroes, unique_comics

(6439, 12651)

In [7]:
# show all nodes with type comic
df_nodes[df_nodes['type'] == 'comic']

,node,type
0,2001 10,comic
1,2001 8,comic
2,2001 9,comic
7,A '00,comic
8,A '01,comic
...,...,...
19021,YOUNG MEN 24,comic
19022,YOUNG MEN 25,comic
19023,YOUNG MEN 26,comic
19024,YOUNG MEN 27,comic


Wir sehen, dass nicht alle comic namen jahren haben

In [8]:
# 1. Überprüfen Sie, ob es Duplikate im `node` Attribut gibt.
duplicate_nodes = df_nodes[df_nodes.duplicated(subset=['node'], keep=False)]

# 2. Überprüfen Sie, ob es Knoten ohne den `type` Attribut gibt oder ob es unerwartete Typwerte gibt.
nodes_without_type = df_nodes[df_nodes['type'].isnull()]
unique_types = df_nodes['type'].unique()

# 3. Überprüfen Sie, ob es Kanten gibt, die zwei Helden oder zwei Comics verbinden.
# Zuerst erstellen wir ein Set von Helden und Comics
heroes_set = set(df_nodes[df_nodes['type'] == 'hero']['node'])
comics_set = set(df_nodes[df_nodes['type'] == 'comic']['node'])

# Überprüfen Sie, ob beide Enden der Kante im Helden-Set oder im Comic-Set sind
invalid_edges_hero_hero = df_edges[(df_edges['hero'].isin(heroes_set)) & (df_edges['comic'].isin(heroes_set))]
invalid_edges_comic_comic = df_edges[(df_edges['hero'].isin(comics_set)) & (df_edges['comic'].isin(comics_set))]

# Zusammenfassung
{
    "duplicate_nodes": duplicate_nodes.shape[0],
    "nodes_without_type": nodes_without_type.shape[0],
    "unique_types": list(unique_types),
    "invalid_edges_hero_hero": invalid_edges_hero_hero.shape[0],
    "invalid_edges_comic_comic": invalid_edges_comic_comic.shape[0]
}

{'duplicate_nodes': 0,
 'nodes_without_type': 0,
 'unique_types': ['comic', 'hero'],
 'invalid_edges_hero_hero': 0,
 'invalid_edges_comic_comic': 0}

Basierend auf diesen Ergebnissen scheint die Datenqualität insgesamt gut zu sein, zumindest in Bezug auf die Punkte, die wir überprüft haben.

## NetworkX-Graph erstellen
### Two-Mode

In [9]:
G_two_mode = nx.Graph(name = 'Two Mode Graph')

# Knoten hinzufügen
for index, row in df_nodes.iterrows():
    G_two_mode.add_node(row['node'], type=row['type'])

# Kanten hinzufügen
for index, row in df_edges.iterrows():
    G_two_mode.add_edge(row['hero'], row['comic'])

In [10]:
print_network_information(G_two_mode)

Information for given Graph with name 'Two Mode Graph':
	Graph is directed: False
	Number of nodes: 19091
	Number of edges: 96104


### One-Mode

In [11]:
# Konvertiere das bipartite Netzwerk in ein One-Mode-Netzwerk
G_one_mode = nx.projected_graph(G_two_mode, [node for node, data in G_two_mode.nodes(data=True) if data.get('type') == 'hero'])
G_one_mode.name = 'One Mode Graph'

print_network_information(G_one_mode)

Information for given Graph with name 'One Mode Graph':
	Graph is directed: False
	Number of nodes: 6440
	Number of edges: 171644
